In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df=df.dropna()

In [6]:
x=df.drop('label',axis=1)

In [7]:
y=df['label']

In [8]:
x.shape,y.shape

((18285, 4), (18285,))

In [10]:
from tensorflow.keras.layers import Embedding,Dense,Dropout,LSTM,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [11]:
voc_size=5000

In [12]:
message=x.copy()

In [13]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
message.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\revei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
### data preprocessing

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus =[]
for i in range(0, len(message)):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [18]:
onhot_rep=[one_hot(words,voc_size) for words in corpus]

In [19]:
onhot_rep[0]

[3165, 4772, 76, 931, 3349, 1590, 1997, 2209, 1550, 1659]

In [20]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [21]:
sent_length=20
embeded_docs=pad_sequences(onhot_rep,padding='pre',maxlen=sent_length)

In [22]:
embeded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3165,
       4772,   76,  931, 3349, 1590, 1997, 2209, 1550, 1659])

In [42]:
dim=40
model=Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
len(embeded_docs),y.shape

(18285, (18285,))

In [23]:
import numpy as np
x_final=np.array(embeded_docs)
y_final=np.array(y)

In [24]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [48]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 46ms/step - loss: 0.4804 - accuracy: 0.7726 - val_loss: 0.1938 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 8s 43ms/step - loss: 0.1386 - accuracy: 0.9481 - val_loss: 0.2014 - val_accuracy: 0.9173
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0979 - accuracy: 0.9659 - val_loss: 0.2472 - val_accuracy: 0.9178
Epoch 4/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0668 - accuracy: 0.9773 - val_loss: 0.2939 - val_accuracy: 0.9170
Epoch 5/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0437 - accuracy: 0.9856 - val_loss: 0.3473 - val_accuracy: 0.9049
Epoch 6/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0285 - accuracy: 0.9913 - val_loss: 0.4083 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0185 - accuracy: 0.9941 - val_loss: 0.3969 - val_accuracy: 0.9

In [49]:
y_pred=model.predict_classes(x_test)


c:\users\revei\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\engine\sequential.py:459: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [50]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [51]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

[[3128  291]
 [ 263 2353]]
0.9082021541010771


# birectional lstm model

In [27]:
dim=40
model1=Sequential()
model1.add(Embedding(voc_size,dim,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
model1.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=64,epochs=10)

Epoch 1/10
192/192 [==============================] - 8s 28ms/step - loss: 0.4430 - accuracy: 0.7524 - val_loss: 0.2019 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1271 - accuracy: 0.9514 - val_loss: 0.2027 - val_accuracy: 0.9150
Epoch 3/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0815 - accuracy: 0.9724 - val_loss: 0.2428 - val_accuracy: 0.9196
Epoch 4/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0480 - accuracy: 0.9850 - val_loss: 0.2617 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0313 - accuracy: 0.9915 - val_loss: 0.3248 - val_accuracy: 0.9022
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0228 - accuracy: 0.9937 - val_loss: 0.3666 - val_accuracy: 0.9080
Epoch 7/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0145 - accuracy: 0.9971 - val_loss: 0.4825 - val_accuracy: 0.9097

In [29]:
y_pred1=model1.predict_classes(x_test)

c:\users\revei\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\engine\sequential.py:459: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [31]:
print(classification_report(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(accuracy_score(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

[[3107  312]
 [ 221 2395]]
0.9116818558409279
